In [20]:
import dotenv
dotenv.load_dotenv()

#pptx transformation imports
import zipfile
import shutil
import os

# translator import
import xml.etree.ElementTree as ET
from openai import OpenAI
from typing import List, Tuple

In [21]:
class pptx_transformer:
    def __init__(self, pptx_path: str,):
        self.pptx_path = pptx_path

def extract_pptx(pptx_path: str, extract_path: str) -> str:
    """
    Extract a PPTX file into its XML components.
    
    Args:
        pptx_path: Path to the PPTX file
        extract_path: Directory where to extract the contents
        
    Returns:
        Path to the extracted directory
    """
    # Create extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)
    
    # Extract the PPTX (which is actually a ZIP file)
    with zipfile.ZipFile(pptx_path, 'r') as pptx:
        pptx.extractall(extract_path)
    
    return extract_path

def compose_pptx(source_path: str, output_pptx: str):
    """
    Compose a PPTX file from a directory containing the XML structure.
    
    Args:
        source_path: Path to the directory containing the PPTX contents
        output_pptx: Path where to save the resulting PPTX file
    """
    # Make sure the output directory exists
    os.makedirs(os.path.dirname(output_pptx), exist_ok=True)
    
    # Create a ZIP file with PPTX extension
    shutil.make_archive(
        output_pptx.replace('.pptx', ''), # Base name without extension
        'zip',                            # Archive format
        source_path                       # Source directory
    )
    
    # Rename the zip to pptx if necessary
    zip_path = output_pptx.replace('.pptx', '.zip')
    if os.path.exists(zip_path):
        shutil.move(zip_path, output_pptx)



In [24]:
import os
import xml.etree.ElementTree as ET
from openai import OpenAI
from typing import List, Tuple

class SlideTranslator:
    def __init__(self, target_language: str, GenAI_api_key: str):
        self.target_language = target_language
        self.client = OpenAI(api_key=GenAI_api_key)
        self.namespaces = {
        'a': 'http://schemas.openxmlformats.org/drawingml/2006/main'
    }

    def find_slide_files(self, root_folder: str) -> List[str]:
        """Find all slide XML files in the folder structure. Schema: "slide<number>.xml" """
        slide_files = []
        for root, _, files in os.walk(root_folder):
            for file in files:
                # Check if file matches pattern: slide<number>.xml
                if file.startswith('slide') and file.endswith('.xml'):
                    # Extract the middle part between 'slide' and '.xml'
                    number_part = file[5:-4]  # Remove 'slide' prefix and '.xml' suffix
                    # Check if the remaining part is a valid integer
                    if number_part.isdigit():
                        slide_files.append(os.path.join(root, file))
        return sorted(slide_files)  # Sort to maintain slide order

    def extract_text_runs(self, xml_file: str) -> List[Tuple[List[ET.Element], str]]:
        """Extract text runs while preserving structure and formatting."""
        tree = ET.parse(xml_file)
        root = tree.getroot()
        paragraphs_data = []
        
        for paragraph in root.findall('.//a:p', self.namespaces):
            paragraph_structure = []
            full_text_parts = []
            
            # Process each run in the paragraph
            for run in paragraph.findall('.//a:r', self.namespaces):
                text_elements = run.findall('.//a:t', self.namespaces)
                for text_elem in text_elements:
                    if text_elem.text and text_elem.text.strip():
                        # Store both the element and its text
                        paragraph_structure.append({
                            'element': text_elem,
                            'text': text_elem.text,
                            'parent_run': run  # Store reference to parent run for formatting
                        })
                        full_text_parts.append(text_elem.text)
            
            if full_text_parts:
                # Store the paragraph data with its structure
                paragraphs_data.append({
                    'structure': paragraph_structure,
                    'full_text': ' '.join(full_text_parts)
                })
        
        return paragraphs_data

    def translate_text(self, text: str) -> str:
        """Translate text while preserving approximate total character length and formatting."""
        prompt = f"""Translate the following text to {self.target_language}. 
        Maintain similar total character length and preserve any special formatting or technical terms.
        Original text: {text}"""
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a professional translator."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3  # Lower temperature for more consistent translations
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Translation error: {e}")
            return text

    def align_translation(self, original_parts: List[dict], translated_text: str) -> List[str]:
        """Use LLM to align translated text with original structure."""
        prompt = f"""Given the original text split into parts and its translation, 
        match each translated segment to the corresponding original part while preserving meaning.
        
        Original parts: {[part['text'] for part in original_parts]}
        Complete translation: {translated_text}
        
        Return the translation split into exactly {len(original_parts)} parts, maintaining the same structure.
        Format your response as a Python list of strings, one for each part."""
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a professional translator helping to align translated text with original structure."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )
            # Parse the response as a Python list
            aligned_parts = eval(response.choices[0].message.content.strip())
            return aligned_parts
        except Exception as e:
            print(f"Alignment error: {e}")
            # Fallback: split translation into equal parts
            return [translated_text] * len(original_parts)

    def process_slides(self, folder_path: str):
        """Main function to process all slides in the presentation."""
        slide_files = self.find_slide_files(folder_path)
        
        for slide_file in slide_files:
            print(f"\nProcessing {os.path.basename(slide_file)}...")
            tree = ET.parse(slide_file)
            
            # Extract paragraphs with their structure
            paragraphs_data = self.extract_text_runs(slide_file)
            
            # Process each paragraph
            for paragraph_data in paragraphs_data:
                original_structure = paragraph_data['structure']
                original_text = paragraph_data['full_text']
                
                if original_text.strip():
                    print(f"Original text: {original_text}")
                    # Get complete translation
                    translated_text = self.translate_text(original_text)
                    print(f"Translated text: {translated_text}")
                    
                    # Align translation with original structure
                    aligned_translations = self.align_translation(original_structure, translated_text)
                    
                    # Update each element with its aligned translation
                    for part, translation in zip(original_structure, aligned_translations):
                        part['element'].text = translation
            
            # Save the modified XML
            tree.write(slide_file, encoding="UTF-8", xml_declaration=True)


In [26]:
# Example usage:
if __name__ == "__main__":
    openai_api_key = os.getenv("OPENAI_API_KEY")
    folder_path = "Presentation1"
    # Extract the PPTX
    pptx_file = "/Users/jwh/Code/Translator/Presentation1.pptx"
    extract_folder = "/Users/jwh/Code/Translator/extracted_pptx"
    extracted_path = extract_pptx(pptx_file, extract_folder)
    
    #Initialize translator and process slides
    translator = SlideTranslator(target_language="German", GenAI_api_key=openai_api_key)
    translator.process_slides(extracted_path)
    
    # Recompose the PPTX
    output_file = "/Users/jwh/Code/Translator/translated_presentation2.pptx"
    compose_pptx(extracted_path, output_file)

    print("done")


Processing slide1.xml...
Original text: This is the presentation title
Translated text: Dies ist der Präsentationstitel
Original text: Second  textblock
Translated text: Zweiter Textblock

Processing slide2.xml...
Original text: This is the title of the second page
Translated text: Dies ist der Titel der zweiten Seite
Original text: Random text as a bullet point list
Translated text: Originaltext: Zufälliger Text als Aufzählungsliste
Original text: Second  bulletpoint  entry
Translated text: Zweiter  Aufzählungspunkt  Eintrag
Original text: Another text in another field on the second page
Translated text: Ein weiterer Text in einem anderen Bereich auf der zweiten Seite

Processing slide3.xml...
Original text: Small and  big  small  and big again  for sure
Translated text: Klein und groß klein und wieder groß sicherlich
